# Results
Compare results and create visualization

In [1]:
import os
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import scipy.stats as stats

Set path to the pickle result files

In [3]:
PATH = "D:/3xM/3xM_Test_Dataset/RESULTS/In-Distribution Experiment"    # Sim-to-real Experiment, In-Distribution Experiment
os.listdir(PATH)

['mask_rcnn_rgb_3xM_Dataset_10_160_epoch_050_eval.pkl',
 'mask_rcnn_rgb_3xM_Dataset_160_160_epoch_050_eval.pkl',
 'mask_rcnn_rgb_3xM_Dataset_80_160_epoch_050_eval.pkl']

Loading all results

In [6]:
results = dict()
for cur_file in os.listdir(PATH):
    cur_path = os.path.join(PATH, cur_file)
    cur_name = ".".join(cur_file.split(".")[:-1])

    # get the result
    if cur_path.endswith(".pkl"):
        with open(cur_path, "rb") as file:  
            loaded_dict = pickle.load(file)
        results[cur_name] = dict()
        for key, value in loaded_dict.items():
            results[cur_name][key] = value

In [9]:
for model_name, metrics in results.items():
    print(f"{model_name}:\n         -> {metrics['intersection over union']}")

mask_rcnn_rgb_3xM_Dataset_10_160_epoch_050_eval:
         -> [0.9577720236480222, 0.8197242184217552, 0.8088515247638409, 0.924293388709994, 0.7521584015688361, 0.8258636640831037, 0.8213785697951185, 0.8222644146610935, 0.8882533413255759, 0.830643064747978, 0.706330655108127, 0.747361433141486, 0.663374583398345, 0.9234634830014726, 0.7825435124500892, 0.9379047322477709, 0.8903003444801718, 0.8615576802504207, 0.8775108657000534, 0.7932950844586473, 0.9324223584470626, 0.8127605751713822, 0.9288179155078103, 0.8908661698193436, 0.7698289873195, 0.8037860279465845, 0.788009922057922, 0.9652138052317749, 0.954840550151863, 0.9346270766742946, 0.8069988328879351, 0.7186253402668634, 0.7590903673143939, 0.8211617333239474, 0.9097853560839297, 0.7430323649527416, 0.7872231879362278, 0.811015952461034, 0.9078482640374466, 0.8223329707746012, 0.7843635600731056, 0.9314978714547003, 0.9228981850909814, 0.813987525136215, 0.7883604308567111, 0.7892137160658123, 0.5122025234896109, 0.74492896

### Compare Significants of change 

Test of normal distribution

In [ ]:
normality_results = {}
for name, data in results.items():
    iou_values = data.get("intersection over union")

    if iou_values:
        shapiro_stat, shapiro_p = stats.shapiro(iou_values)
        normality_results[name] = (shapiro_stat, shapiro_p)
        if shapiro_p > 0.05:
            print(f"{name}: IOU-values are normally distributed (p-Wert = {shapiro_p:.3f})")
        else:
            print(f"{name}: IOU-values are NOT normally distributed (p-Wert = {shapiro_p:.3f})")

all_normal = all(p > 0.05 for _, p in normality_results.values())
if all_normal:
    print("\n\n    -> All results are normally distributed, t test will be applied.")
else:
    print("\n\n    -> NOT All results are normally distributed, Wilcoxon test will be applied.")

Let the test beginn

In [ ]:
keys = list(results.keys())
num_results = len(keys)
p_values_matrix = np.ones((num_results, num_results))  # Initialisiere mit 1 (Diagonale wird sp�ter 0)

# pairwaise t test
for i in range(num_results):
    for j in range(i + 1, num_results):
        iou_1 = results[keys[i]]["intersection over union"]
        iou_2 = results[keys[j]]["intersection over union"]
        
        # making a t test o wilcoxon test
        if all_normal:
            _, p_value = stats.ttest_rel(iou_1, iou_2)
        else:
            _, p_value = stats.wilcoxon(iou_1, iou_2)

        p_values_matrix[i, j] = p_value

# plotting of p values of all paris
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(p_values_matrix, dtype=bool)) 
sns.heatmap(p_values_matrix, mask=mask, annot=True, fmt=".3f", cmap="coolwarm", 
            xticklabels=keys, yticklabels=keys, cbar_kws={"label": "p-Wert"})

plt.title("P-Values test of IOU significants")
plt.show()

Here is a detailed explanation for interpreting the results from the half heatmap plot of p-values:

1. **Understanding p-values in the Context of Statistical Significance**:
   - Each cell in the heatmap represents the p-value from a pairwise statistical test comparing the Intersection over Union (IOU) results of two experiments.
   - A **p-value** indicates the probability that the observed difference between two sets of IOU results is due to random chance. Lower p-values suggest a significant difference, while higher p-values suggest that the difference may not be meaningful.

2. **Heatmap Color Coding**:
   - **Dark Colors (Low p-values)**: These cells represent pairs of experiments where the IOU results are significantly different. For example, if a cell has a p-value < 0.05, it means that there is less than a 5% chance that the observed difference is random. This suggests that the experimental modifications in those pairs likely lead to a real change in IOU performance.
   - **Light Colors (High p-values)**: Cells with higher p-values (e.g., > 0.05) indicate pairs of experiments where the difference in IOU is not statistically significant. In other words, the observed difference could likely be due to random variation, and there is not enough evidence to conclude a true performance difference.

3. **Diagonal and Masked Elements**:
   - The **diagonal cells** (where each experiment is compared to itself) are masked because they naturally have a p-value of 1.0 (no difference exists when comparing a result to itself).
   - The **upper half** of the matrix is masked to simplify the plot, as the p-values are symmetrical; each pairwise comparison only needs to be shown once.

4. **Choosing Significance Thresholds**:
   - Typically, a **threshold of 0.05** is used to denote significance (darker colors in cells below this threshold), but some researchers may apply more stringent thresholds (e.g., 0.01 or 0.001) depending on their confidence requirements.
   - If a significant number of cells show p-values below 0.05, it suggests that many experimental adjustments may genuinely affect IOU performance. Conversely, if most p-values are above 0.05, it may indicate that the experimental variations do not lead to meaningful differences in IOU performance.

5. **Further Considerations and Practical Implications**:
   - A **cluster of dark cells** for certain experiments suggests that those specific experiments may have systematically different IOU results compared to others. This can be valuable in identifying which experimental adjustments are particularly impactful.
   - The presence of high p-values (light cells) between multiple experiments implies that those experimental variations produce similar IOU results, indicating that changing those factors may not significantly impact model performance.
   - **Interpret the direction and magnitude** of significant differences separately from the p-values, as the heatmap itself only tells us whether the difference is statistically significant, not whether it improves or worsens IOU.

By using this heatmap interpretation, you can effectively determine which experiments lead to meaningful changes in performance, guiding future adjustments or fine-tuning based on significant outcomes.

### Visualize Results

In [ ]:
# Calculate mean IOU for each experiment
mean_results = {name: np.mean(data["intersection over union"]) for name, data in results.items()}

# Plot the mean results in a bar chart using subplots
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(mean_results.keys(), mean_results.values(), color="skyblue")
ax.set_xlabel("Experiment")
ax.set_ylabel("Mean IOU")
ax.set_title("Mean IOU Results by Experiment")
ax.set_xticklabels(mean_results.keys(), rotation=45, ha="right")

# Show the plot
plt.tight_layout()
plt.show()

... more visualizations

---